In [26]:
import pandas as pd
import numpy as np

In [36]:
data_path = 'E:/YandexDisk/DNAm draft/unn_levine/'
data_file = 'Долгожители (L) и Q.xlsx'

df = pd.read_excel(f'{data_path}{data_file}', sheet_name='Испытуемые')

In [38]:
new_df = df[['Код', 'Год рождения', 'Дата забора', 'альбумин (g/L)', 'креатинин (umol/L)', 'глюкоза (mmol/L)', 'СРБ (мг/л)', 'LYM, %', 'MCV,fl', 'RDW-CV,%', 'ЩФ (U/L)', 'WBC, 10^9л']]
new_df.dropna(inplace=True)
new_df['Age'] = (new_df['Дата забора'] - new_df['Год рождения']) / np.timedelta64(1, 'D') / 365.25
new_df.drop(columns=['Дата забора', 'Год рождения'], inplace=True)
new_df = new_df.rename(columns={'Код': 'ID', 'age': 'Age',
                                'альбумин (g/L)': 'Albumin', 'креатинин (umol/L)': 'Creatinine', 
                                'глюкоза (mmol/L)': 'Glucose', 'СРБ (мг/л)': 'C_reactive_protein', 
                                'LYM, %': 'Lymphocyte_percent', 'MCV,fl': 'Mean_cell_volume', 
                                'RDW-CV,%': 'Red_cell_distribution_width', 'ЩФ (U/L)': 'Alkaline_phosphatase', 
                                'WBC, 10^9л': 'White_blood_cell_count'})


C:\Users\alena\AppData\Local\Temp\ipykernel_4008\956121368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df.dropna(inplace=True)
C:\Users\alena\AppData\Local\Temp\ipykernel_4008\956121368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['Age'] = (new_df['Дата забора'] - new_df['Год рождения']) / np.timedelta64(1, 'D') / 365.25
C:\Users\alena\AppData\Local\Temp\ipykernel_4008\956121368.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org

In [39]:
data_dict = {}
data_dict['Albumin'] = list(new_df.Albumin)
data_dict['Creatinine'] = list(new_df.Creatinine)
data_dict['Glucose'] = list(new_df.Glucose)
data_dict['C_reactive_protein'] = np.log10(list(new_df.C_reactive_protein * 10))
data_dict['Lymphocyte_percent'] = list(new_df.Lymphocyte_percent)
data_dict['Mean_cell_volume'] = list(new_df.Mean_cell_volume)
data_dict['Red_cell_distribution_width'] = list(new_df.Red_cell_distribution_width)
data_dict['Alkaline_phosphatase'] = list(new_df.Alkaline_phosphatase)
data_dict['White_blood_cell_count'] = list(new_df.White_blood_cell_count)
data_dict['Age'] = list(new_df.Age)

coeff_dict = {}
coeff_dict['Albumin'] = -0.0336
coeff_dict['Creatinine'] = 0.0095
coeff_dict['Glucose'] = 0.1953
coeff_dict['C_reactive_protein'] = 0.0954
coeff_dict['Lymphocyte_percent'] = -0.0120
coeff_dict['Mean_cell_volume'] = 0.0268
coeff_dict['Red_cell_distribution_width'] = 0.3306
coeff_dict['Alkaline_phosphatase'] = 0.0019
coeff_dict['White_blood_cell_count'] = 0.0554
coeff_dict['Age'] = 0.0804
const = -19.9067
gamma = 0.0077

linear_combinations = np.zeros(len(data_dict['Age']))
mortality_score = np.zeros(len(data_dict['Age']))
mortality_score_1_year = np.zeros(len(data_dict['Age']))
mortality_score_2_year = np.zeros(len(data_dict['Age']))
phenotypic_age = np.zeros(len(data_dict['Age']))

for sub_id in range(0, len(data_dict['Age'])):

    for key in coeff_dict:
        linear_combinations[sub_id] += coeff_dict[key] * data_dict[key][sub_id]
    linear_combinations[sub_id] += const

    mortality_score[sub_id] = 1 - np.exp(-np.exp(linear_combinations[sub_id]) * (np.exp(120 * gamma) - 1) / gamma)

    mortality_score_1_year[sub_id] = 1 - np.exp(-np.exp(linear_combinations[sub_id]) * (np.exp(12 * gamma) - 1) / gamma)
    mortality_score_2_year[sub_id] = 1 - np.exp(-np.exp(linear_combinations[sub_id]) * (np.exp(24 * gamma) - 1) / gamma)

    phenotypic_age[sub_id] = 141.50225 + np.log(-0.00553 * np.log(1 - mortality_score[sub_id])) / 0.090165


C:\Users\alena\AppData\Local\Temp\ipykernel_4008\1652312954.py:5: RuntimeWarning: divide by zero encountered in log10
  data_dict['C_reactive_protein'] = np.log10(list(new_df.C_reactive_protein * 10))
C:\Users\alena\AppData\Local\Temp\ipykernel_4008\1652312954.py:44: RuntimeWarning: divide by zero encountered in log
  phenotypic_age[sub_id] = 141.50225 + np.log(-0.00553 * np.log(1 - mortality_score[sub_id])) / 0.090165


In [41]:
new_df['PhenoAge'] = phenotypic_age
new_df.to_excel(f'{data_path}unn_levine_hui.xlsx')